<a href="https://colab.research.google.com/github/A7mad-Salem/ANLP-Project./blob/main/ANLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy --upgrade --force-reinstall


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 111.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.


In [ ]:
import re
import pandas as pd
import torch
from torch.nn.utils.rnn import pad_sequence
import numpy as np


In [ ]:
data=pd.read_csv("/content/drive/MyDrive/ANLP_project/validation.csv")

In [ ]:
data=data[0:10000]
data.head()

,id,article,highlights
0,61df4979ac5fcc2b71be46ed6fe5a46ce7f071c3,"Sally Forrest, an actress-dancer who graced th...","Sally Forrest, an actress-dancer who graced th..."
1,21c0bd69b7e7df285c3d1b1cf56d4da925980a68,A middle-school teacher in China has inked hun...,Works include pictures of Presidential Palace ...
2,56f340189cd128194b2e7cb8c26bb900e3a848b4,A man convicted of killing the father and sist...,"Iftekhar Murtaza, 29, was convicted a year ago..."
3,00a665151b89a53e5a08a389df8334f4106494c2,Avid rugby fan Prince Harry could barely watch...,Prince Harry in attendance for England's crunc...
4,9f6fbd3c497c4d28879bebebea220884f03eb41a,A Triple M Radio producer has been inundated w...,Nick Slater's colleagues uploaded a picture to...


In [ ]:
data=data.drop(columns='id')
# Rename the column 'highlights'
data.rename(columns={'highlights': 'Summary','article':'Article'}, inplace=True)
data.head()

,Article,Summary
0,"Sally Forrest, an actress-dancer who graced th...","Sally Forrest, an actress-dancer who graced th..."
1,A middle-school teacher in China has inked hun...,Works include pictures of Presidential Palace ...
2,A man convicted of killing the father and sist...,"Iftekhar Murtaza, 29, was convicted a year ago..."
3,Avid rugby fan Prince Harry could barely watch...,Prince Harry in attendance for England's crunc...
4,A Triple M Radio producer has been inundated w...,Nick Slater's colleagues uploaded a picture to...


In [ ]:
# Define a text cleaning function
def clean_text(text):
    if pd.isnull(text):
        return ""

    text = text.lower()  # Convert to lowercase
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags

    # Optionally keep digits and basic punctuation (e.g., .,!?)
    text = re.sub(r'[^a-z0-9\s\.\,\!\?]', '', text)

    # Normalize spaces (remove multiple spaces)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Apply the preprocessing function to the Article and Summary columns
data['clean_Article'] = data['Article'].apply(clean_text)
data['clean_summary'] = data['Summary'].apply(clean_text)

print('Processing Done. 🎉🎉🎉')

Processing Done. 🎉🎉🎉


In [ ]:
data.head()

,Article,Summary,clean_Article,clean_summary
0,"Sally Forrest, an actress-dancer who graced th...","Sally Forrest, an actress-dancer who graced th...","sally forrest, an actressdancer who graced the...","sally forrest, an actressdancer who graced the..."
1,A middle-school teacher in China has inked hun...,Works include pictures of Presidential Palace ...,a middleschool teacher in china has inked hund...,works include pictures of presidential palace ...
2,A man convicted of killing the father and sist...,"Iftekhar Murtaza, 29, was convicted a year ago...",a man convicted of killing the father and sist...,"iftekhar murtaza, 29, was convicted a year ago..."
3,Avid rugby fan Prince Harry could barely watch...,Prince Harry in attendance for England's crunc...,avid rugby fan prince harry could barely watch...,prince harry in attendance for englands crunch...
4,A Triple M Radio producer has been inundated w...,Nick Slater's colleagues uploaded a picture to...,a triple m radio producer has been inundated w...,nick slaters colleagues uploaded a picture to ...


In [ ]:
#  Show a sample
print("Original Article:\n", data['Article'][0])
print("\nCleaned Article:\n", data['clean_Article'][0])
print("\nOriginal summary:\n", data['Summary'][0])
print("\nCleaned summary:\n", data['clean_summary'][0])

Original Article:
 Sally Forrest, an actress-dancer who graced the silver screen throughout the '40s and '50s in MGM musicals and films such as the 1956 noir While the City Sleeps died on March 15 at her home in Beverly Hills, California. Forrest, whose birth name was Katherine Feeney, was 86 and had long battled cancer. Her publicist, Judith Goffin, announced the news Thursday. Scroll down for video . Actress: Sally Forrest was in the 1951 Ida Lupino-directed film 'Hard, Fast and Beautiful' (left) and the 1956 Fritz Lang movie 'While the City Sleeps' A San Diego native, Forrest became a protege of Hollywood trailblazer Ida Lupino, who cast her in starring roles in films including the critical and commercial success Not Wanted, Never Fear and Hard, Fast and Beautiful. Some of Forrest's other film credits included Bannerline, Son of Sinbad, and Excuse My Dust, according to her iMDB page. The page also indicates Forrest was in multiple Climax! and Rawhide television episodes. Forrest app

In [ ]:
from transformers import T5Tokenizer
from torch.utils.data import Dataset, DataLoader, random_split
import torch

# Load T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Apply "summarize:" prefix to the *cleaned* article, not raw
def add_prefix(text):
    return "summarize: " + text

# Apply prefix to the cleaned articles
data['clean_Article'] = data['clean_Article'].apply(add_prefix)

# Tokenization function
def tokenize_data(examples):
    # Tokenize input (article)
    model_inputs = tokenizer(
        examples['clean_Article'],
        max_length=512,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
        return_attention_mask=True
    )

    # Tokenize target (summary)
    labels = tokenizer(
        examples['clean_summary'],
        max_length=64,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    ).input_ids

    # Replace padding token id in labels with -100 to ignore in loss
    labels[labels == tokenizer.pad_token_id] = -100

    # Add labels to the model inputs
    model_inputs["labels"] = labels
    return model_inputs

# Prepare inputs as lists
tokenized_dataset = tokenize_data({
    'clean_Article': data['clean_Article'].tolist(),
    'clean_summary': data['clean_summary'].tolist()
})

print("✅ Tokenization complete.")
print("Input shape:", tokenized_dataset['input_ids'].shape)
print("Label shape:", tokenized_dataset['labels'].shape)

✅ Tokenization complete.
Input shape: torch.Size([10000, 512])
Label shape: torch.Size([10000, 64])


In [ ]:
from torch.utils.data import Dataset

class TextSummaryDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }


In [ ]:
# Create Dataset object
dataset = TextSummaryDataset(
    input_ids=tokenized_dataset['input_ids'],
    attention_mask=tokenized_dataset['attention_mask'],
    labels=tokenized_dataset['labels']
)

# Split into train and validation (80% / 20%)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=4)

print(f"✅ DataLoaders ready: {len(train_loader)} train batches, {len(val_loader)} val batches.")

✅ DataLoaders ready: 2000 train batches, 500 val batches.


In [ ]:
!pip install transformers

In [ ]:
from transformers import T5Config, T5ForConditionalGeneration

# Custom config with dropout
config = T5Config.from_pretrained("t5-small")
config.dropout_rate = 0.3
config.dropout = 0.3

# Load model with config
model = T5ForConditionalGeneration.from_pretrained("t5-small", config=config)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [ ]:
from torch.optim import AdamW

# Initialize the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)
  # slightly higher LR speeds up convergence


In [ ]:
from tqdm import tqdm
import torch
import matplotlib.pyplot as plt

train_losses = []
val_losses = []
patience=5
trials=0

n_epochs = 30
best_val_loss = float('inf')

In [ ]:
from transformers import get_scheduler

num_training_steps = len(train_loader) * n_epochs
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)


In [ ]:
print("🚀 Start model training...")

for epoch in range(1, n_epochs + 1):
    model.train()
    epoch_train_loss = 0

    with tqdm(total=len(train_loader), desc=f"Epoch {epoch}/{n_epochs}") as pbar:
        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            loss = outputs.loss
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            lr_scheduler.step()

            epoch_train_loss += loss.item()
            pbar.set_postfix(loss=loss.item())
            pbar.update(1)

    avg_train_loss = epoch_train_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Evaluation
    model.eval()
    epoch_val_loss = 0

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            val_loss = outputs.loss
            epoch_val_loss += val_loss.item()

    avg_val_loss = epoch_val_loss / len(val_loader)
    val_losses.append(avg_val_loss)

    print(f"📢 Epoch {epoch}: Train Loss = {avg_train_loss:.4f}, Val Loss = {avg_val_loss:.4f}")

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), "best_t5_model.pth")
        print(f"✅ Best model saved at epoch {epoch} with Val Loss: {best_val_loss:.4f}")
        trials = 0  # reset counter
    else:
        trials += 1
        if trials >= patience:
            print(f"⏹️ Early stopping at epoch {epoch}")
            break



🚀 Start model training...


Epoch 1/30: 100%|██████████| 2000/2000 [05:57<00:00,  5.59it/s, loss=2.91]


📢 Epoch 1: Train Loss = 2.8550, Val Loss = 2.1032
✅ Best model saved at epoch 1 with Val Loss: 2.1032


Epoch 2/30: 100%|██████████| 2000/2000 [05:42<00:00,  5.83it/s, loss=2.14]


📢 Epoch 2: Train Loss = 2.6737, Val Loss = 2.1337


...

Epoch 15/30: 100%|██████████| 2000/2000 [05:41<00:00,  5.85it/s, loss=1.95]


📢 Epoch 15: Train Loss = 2.3663, Val Loss = 2.0880


Epoch 16/30: 100%|██████████| 2000/2000 [05:41<00:00,  5.85it/s, loss=2.05]


📢 Epoch 16: Train Loss = 2.3577, Val Loss = 2.0951


Epoch 17/30:  66%|██████▌   | 1319/2000 [03:45<01:56,  5.86it/s, loss=2.12]

In [ ]:
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training & Validation Loss")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
!pip install evaluate

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import evaluate


# Load model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("t5-small")
model.load_state_dict(torch.load("best_t5_model.pth", map_location=torch.device('cpu')))
model.eval()

tokenizer = T5Tokenizer.from_pretrained("t5-small")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Load ROUGE metric
rouge = evaluate.load("rouge")

# Function to generate predictions
def generate_summary(text):
    input_ids = tokenizer("summarize: " + text, return_tensors="pt", truncation=True, padding="max_length", max_length=512).input_ids.to(device)
    outputs = model.generate(input_ids=input_ids, max_length=64, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Evaluate on validation set
predictions = []
references = []

for i in range(100):  # Small sample of the val_dataset
    item = val_dataset[i]
    input_text = tokenizer.decode(item['input_ids'], skip_special_tokens=True).replace("summarize: ", "")

    reference_summary = tokenizer.decode(
        [t for t in item['labels'].tolist() if t != -100],  # Filter out -100
        skip_special_tokens=True
    )

    pred_summary = generate_summary(input_text)

    predictions.append(pred_summary)
    references.append(reference_summary)

# Compute ROUGE scores
results = rouge.compute(predictions=predictions, references=references, use_stemmer=True)

# Print ROUGE-1, ROUGE-2, and ROUGE-L
print("📊 ROUGE-1:", results["rouge1"])
print("📊 ROUGE-2:", results["rouge2"])
print("📊 ROUGE-L:", results["rougeL"])


In [ ]:
!pip install bert_score

In [ ]:
# Load BERTScore metric
bertscore = evaluate.load("bertscore")

# Compute BERTScore
bertscore_result = bertscore.compute(
    predictions=predictions,
    references=references,
    lang="en"
)

# Print average BERTScore F1
average_f1 = sum(bertscore_result["f1"]) / len(bertscore_result["f1"])
print("🤖 BERTScore F1 (avg):", average_f1)


In [ ]:
for i in range(5):  # Show 5 example summaries
    item = val_dataset[i]
    input_text = tokenizer.decode(item['input_ids'], skip_special_tokens=True).replace("summarize: ", "")
    reference_summary = tokenizer.decode(
        [t for t in item['labels'].tolist() if t != -100],
        skip_special_tokens=True
    )
    predicted_summary = generate_summary(input_text)

    print(f"\n🔹 Example {i+1}")
    print("📥 Original Text:\n", input_text[:250] + "..." if len(input_text) > 250 else input_text)
    print("✅ Reference Summary:\n", reference_summary)
    print("✏️ Predicted Summary:\n", predicted_summary)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("t5-small")

inputs = data['Article'].tolist()
input_lengths = [len(tokenizer.encode(text, truncation=True)) for text in inputs]

print("📏 Average Input Length:", np.mean(input_lengths))

plt.figure(figsize=(8, 6))  # Adjust figsize as needed
sns.histplot(input_lengths, bins=30, color="skyblue", label="Input", kde=True)
plt.title("Input Length Distribution (Token Count)")
plt.xlabel("Number of Tokens")
plt.ylabel("Frequency")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
reference_lengths = [len(tokenizer.encode(ref, truncation=True)) for ref in references]
predicted_lengths = [len(tokenizer.encode(pred, truncation=True)) for pred in predictions]

print("📏 Average Reference Summary Length:", np.mean(reference_lengths))
print("📏 Average Predicted Summary Length:", np.mean(predicted_lengths))

plt.figure(figsize=(8, 6))  # Adjust figsize as needed
sns.histplot(reference_lengths, bins=30, color="lightgreen", label="Reference", kde=True)
sns.histplot(predicted_lengths, bins=30, color="salmon", label="Predicted", kde=True)
plt.title("Reference and Predicted Summary Length Distributions (Token Count)")
plt.xlabel("Number of Tokens")
plt.ylabel("Frequency")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# Load model and tokenizer
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
model.load_state_dict(torch.load("best_t5_model.pth", map_location=torch.device('cuda')))
model.eval()

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Function to summarize text
def summarize(text):
    input_text = "summarize: " + text
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
    summary_ids = model.generate(inputs, max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return output

# Build GUI
interface = gr.Interface(
    fn=summarize,
    inputs=gr.Textbox(lines=10, placeholder="Enter your text here...", label="Input Text"),
    outputs=gr.Textbox(label="Summary",lines=10,placeholder="Output..."),
    title="📝 Text Summarizer using T5-small",
    description="Enter a long piece of text and click 'Submit' to get the summary.",
    theme="soft"  # You can try "default", "huggingface", or others
)

# Launch GUI
interface.launch()
